In [ ]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data
    
def json_print(data):
    print(json.dumps(data, indent=2))
    
json_print(data)

In [ ]:
import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]  # Replace this with your actual key
    }
)

In [ ]:
if client.schema.exists("Question"):
    client.schema.delete_class("Question")

In [ ]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
}

client.schema.create_class(class_obj)

In [ ]:
with client.batch.configure() as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

In [ ]:
json_print(client.query.aggregate('Question').with_meta_count().do())

### Lets run a vector search to see whats comes back

In [ ]:
#Write a vector search related to animals

# ADD CODE HERE

json_print(response)

### No we want to pass each of these objects to a LLM individually to use when answering a prompt!

In [ ]:
#Write a prompt that will be passed in the returend object above.

# ADD CODE HERE

In [ ]:
#Write a query to perform RAG

# ADD CODE HERE

json_print(response)

### Lets extract all the categories

In [ ]:
response = (client.query
            .get("Question", 'category')
            .with_near_text({'concepts':'animals'})
            .with_limit(10)
            .do()
           )

json_print(response)

### Now we'll pass all of these in at the same time for a LLM to generate a grouped answer.

In [ ]:
#Write a prompt that requires information from all returned objects

# ADD CODE HERE

In [ ]:
#write a query that generates a grouped response

# ADD CODE HERE

json_print(response)